In [ ]:
import pandas as pd
import requests
from openpyxl import load_workbook

# === Chemin du fichier ===
fichier_excel = "donnees_solaire.xlsx"

# --- Lire les données avec pandas ---
df = pd.read_excel(fichier_excel, sheet_name="Sites étudiés", header=1)  # header=1 si noms de colonnes ligne 2

# --- Fonction pour récupérer la production annuelle ---
def get_pvgis_annual(lat, lon, peak_power_kw=1, loss=3.5, slope=None, azimuth=None, timeout=30):
    url = "https://re.jrc.ec.europa.eu/api/v5_2/PVcalc"
    params = {
        "lat": lat,
        "lon": lon,
        "peakpower": peak_power_kw,
        "loss": loss,
        "optimalinclination": 0,  # pas d'optimisation
        "outputformat": "json",
        "radiation_database": "PVGIS-SARAH3",
        "usehorizon": 1
    }
    if slope is not None:
        params["slope"] = slope
    if azimuth is not None:
        params["azimuth"] = azimuth

    try:
        response = requests.get(url, params=params, timeout=timeout)
        response.raise_for_status()
        data = response.json()
        return data["outputs"]["totals"]["fixed"]["E_y"]
    except Exception as e:
        print(f"⚠️ Erreur PVGIS site ({lat}, {lon}) : {e}")
        return 0

# --- Fonction pour récupérer la production mensuelle ---
def get_pvgis_monthly(lat, lon, peak_power_kw=1, loss=3.5, slope=None, azimuth=None, timeout=30):
    url = "https://re.jrc.ec.europa.eu/api/v5_2/PVcalc"
    params = {
        "lat": lat,
        "lon": lon,
        "peakpower": peak_power_kw,
        "loss": loss,
        "optimalinclination": 0,
        "outputformat": "json",
        "radiation_database": "PVGIS-SARAH3",
        "usehorizon": 1
    }
    if slope is not None:
        params["slope"] = slope
    if azimuth is not None:
        params["azimuth"] = azimuth

    try:
        response = requests.get(url, params=params, timeout=timeout)
        response.raise_for_status()
        data = response.json()
        monthly_data = data["outputs"]["monthly"]["fixed"]
        return [month["E_m"] for month in monthly_data]
    except Exception as e:
        print(f"⚠️ Erreur PVGIS site ({lat}, {lon}) : {e}")
        return [0]*12

# --- Charger le fichier Excel avec openpyxl ---
wb = load_workbook(fichier_excel)
ws = wb["Sites étudiés"]

# --- Identifier les colonnes ---
header_row = 2
headers = {cell.value: cell.column for cell in ws[header_row] if cell.value is not None}

# Colonnes de production annuelle et mensuelle
cols_annual = "P_PVGIS_1kWc"
cols_monthly = ["Prod Janvier","Prod Février","Prod Mars","Prod Avril","Prod Mai","Prod Juin",
                "Prod Juillet","Prod Août","Prod Septembre","Prod Octobre","Prod Novembre","Prod Décembre"]

# --- Boucle sur chaque site ---
for i, row in df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    slope = row["InclinaisonPVGIS"]
    azimuth = row["AzimutPVGIS"]
    p_installable = row["P_installable"]  # Puissance installable réelle en kWp


    # Production annuelle
    prod_annuelle = get_pvgis_annual(lat, lon, peak_power_kw=1, loss=3.5, slope=slope, azimuth=azimuth)
    prod_annuelle *= 1.15 # On augmente les résultats de 15% car différence entre API PVGIS et PVGIS site
    excel_row = i + 3  # ligne de données
    col_letter_annual = ws.cell(row=header_row, column=headers[cols_annual]).column_letter
    ws[f"{col_letter_annual}{excel_row}"].value = round(prod_annuelle, 2)

    # Production mensuelle
    prod_mensuelle = get_pvgis_monthly(lat, lon, peak_power_kw=1, loss=3.5, slope=slope, azimuth=azimuth)
    prod_mensuelle = [x * 1.15 * p_installable for x in prod_mensuelle]  # On augmente les résultats de 15% car différence entre API PVGIS et PVGIS site

    for month_idx, month_col in enumerate(cols_monthly):
        col_letter = ws.cell(row=header_row, column=headers[month_col]).column_letter
        ws[f"{col_letter}{excel_row}"].value = round(prod_mensuelle[month_idx], 2)

# --- Sauvegarde ---
wb.save(fichier_excel)
print("✅ Production annuelle et mensuelle PVGIS mise à jour")
